In [1]:
import logging
import os
import random
import sys
from filelock import FileLock

# __import_lightning_begin__
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F
from torchvision.datasets import FashionMNIST
from torchvision import transforms
# __import_lightning_end__

import torchvision.transforms.functional as TF

from MNISTDataModule import MNISTDataModule
from augmentation import TRANSFORM_NAMES

from LightningFashionMNIST import LightningFashionMNIST

log = logging.getLogger(__name__)


In [2]:
logging.basicConfig(stream=sys.stderr, level=logging.INFO)
log.info("Starting...")

# set up the augmentations
# tuple of augmentation name and its magnitude
augmentations = []
for tfn_name in TRANSFORM_NAMES:
    augmentations.append((tfn_name, random.random()))

conf = {
    "progress_bar_refresh_rate": 25,
    "layer_1_size": 512,
    "layer_2_size": 512,
    "lr": 0.001,
    "batch_size": 32,
    "data_dir": "./data",
    # Fashion mnist mean and std
    "data_mean": 0.28604063391685486,
    "data_std": 0.35302430391311646,
    "augmentations": augmentations,
}

log.info(f"Conf {conf}")

data = MNISTDataModule(conf=conf)

model = LightningFashionMNIST(conf=conf)

trainer = pl.Trainer(
    default_root_dir="./data",
    gpus=-1 if torch.cuda.device_count() > 0 else 0,
    max_epochs=2,
    progress_bar_refresh_rate=conf["progress_bar_refresh_rate"],
    num_sanity_val_steps=0,
)

trainer.fit(model, data)

INFO:__main__:Starting...
INFO:__main__:Conf {'progress_bar_refresh_rate': 25, 'layer_1_size': 512, 'layer_2_size': 512, 'lr': 0.001, 'batch_size': 32, 'data_dir': './data', 'data_mean': 0.28604063391685486, 'data_std': 0.35302430391311646, 'augmentations': [('auto_contrast', 0.12398441354252054), ('blur', 0.1894176861717355), ('rotate', 0.40893021599830826)]}
INFO:MNISTDataModule:batch_size: 32. num_workers: 8. data_dir: ./data.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
INFO:filelock:Lock 140574872481744 acquired on /home/akaver/.data.lock


  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/MNISTDataSet/raw/train-images-idx3-ubyte.gz to ./data/MNISTDataSet/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/MNISTDataSet/raw/train-labels-idx1-ubyte.gz to ./data/MNISTDataSet/raw



...

Extracting ./data/MNISTDataSet/raw/t10k-labels-idx1-ubyte.gz to ./data/MNISTDataSet/raw




  | Name    | Type   | Params
-----------------------------------
0 | layer_1 | Linear | 401 K 
1 | layer_2 | Linear | 262 K 
2 | layer_3 | Linear | 5.1 K 
-----------------------------------
669 K     Trainable params
0         Non-trainable params
669 K     Total params
2.679     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]